Обозначения:

📝 - определение, которое нужно запомнить

`# ⏩`- комментарий, что эту ячейку / функцию нужно просто запустить, она уже написана

`# ✏️` - код в этой ячейке / функции мы будем дополнять в процессе урока

👨🏻‍💻 - задача для самостоятельного выполнения

[Чистая рабочая тетрадь для заполнения](https://colab.research.google.com/drive/1hQ69Nn3nnCF4PpBN2HOxXXDJQQm-mL0i?usp=sharing)

[Заполненная рабочая тетрадь](https://colab.research.google.com/drive/1DOIDJXYM9a-DAGGnHIK4qyZECwg0higW?usp=sharing)

[Презентация к уроку](https://docs.google.com/presentation/d/10m7FYPHtHWrby_vl6Z1a-pkrL8magcM0wIruskHNka4/edit?usp=sharing)

# Чем мы будем заниматься на уроке?

Мы напишем систему для подбора похожих новостей. Возьмем данные о новостях с Lenta.ru, с помощью нейросети Word2Vec превратим текстовые описания в числа и объединим в группы на основе близости чисел.

# Немного теории: концептуальная схема работы нейронной сети

Вспомним, в чем заключается задача Data Scientist:
<table>
  <tr>
    <th></th> <th>Прямая задача</th> <th>Обратная задача</th>
  </tr>
  <tr>
    <th>Дано</th>
    <td>y = f(x)</td>
    <td bgcolor="#93E9BE">Координаты y</td>
  </tr>
  <tr>
    <th>Найти</th>
    <td>Координаты y</td>
    <td bgcolor="#93E9BE">Функцию y = z(x)</td>
  </tr>
  <tr>
    <th></th>
    <th>↑ Этому учат в школе ↑</th>
    <th>↑ <u>Этим занимаются DS</u>↑</th>
  </tr>
</table>

📝 **Нейронная сеть** - тип модели машинного обучения, которая состоит из комбинации нейронов - простых функций вида $y_k = w_1 x_2 + ... + w_i x_i$.

📝 **Веса модели** - коэффициенты $w_1 , \dots,  x_i$

Крутость нейросети в том, с помощью нейронов мы можем приблизительно вычислить практически любую функцию (аналогия: цифровое фото из квадратных пикселей). Любая модель семейства GPT по сути являются функцией, описывающей текст на человеческом языке, т.н. "большая языковая модель".

Один из простейших примеров - вычисление функции XOR(a, b) - исключающего ИЛИ. Аргументы a и b могут быть равны 1 или 0. XOR(a, b) вычисляется по следующему правилу:
* XOR = 0, если a = b,
* XOR = 1, если a != b.

Нейронная сеть будет вычислять эту функцию по следующей схеме:

<table style="border: 1px solid transparent">
    <tr><td><img src='https://drive.google.com/uc?export=view&id=1zn68tWxZpZzWOrtxlL4_MHy8MWFIyrEE' height=200>
    </td></tr>
    <tr><td>Картинка с <a href="https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D1%86%D0%B5%D0%BF%D1%82%D1%80%D0%BE%D0%BD">Википедии</a></td></tr>
</table>


Сложность работы с нейронной сетью - это подбор правильных $w_i$. Для этого используются разные комбинации нейронов и разные типы задач.

Например, чтобы преобразовать текст в числа, модель обучают угадывать замаскированное слово по контексту, а потом берут получившиеся веса.
Например, "Мороз и солнце, [???] чудесный". После получения представления текста в виде чисел из весов модели, мы можем выполнять различные математические преобразования над текстом, например "король − мужчина + женщина = королева" (можно подробнее почитать на [Хабре](https://habr.com/ru/articles/446530/))

# Практика - подбираем релевантные новости с помощью нейросети

Пишем систему для рекомендации похожих новостей.

Наш датасет с новостями имеет темы - **topic** ("Спорт", "Наука и техника", ...) и тэги **tag** ("Футбол", "Техника", "Космос", ...).

1. Возьмем новости, помеченные одним тэгом и разобьем на еще более маленькие группы по похожести текстов.

2. Используем библиотеку, реализующую нейросеть архитектуры Word2Vec от [gensim](https://pypi.org/project/gensim/) и готовые веса для нее от [RusVectores](https://rusvectores.org/ru/models/). Преобразуем тексты в наборы чисел.

3. Используем модели для 📝 **кластеризации** - разбиения набора данных на группы наиболее похожих элементов. Воспользуемся реализацией из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

In [1]:
# ▶️▶️
# Подключаем нужные библиотеки для визуализации

import nltk
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans

nltk.download('punkt') # загружаем дополнительные данные для библиотеки разбиения текстов на слова

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\toxidxd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# ▶️▶️
# Настраиваем отображение таблиц
from IPython.display import display
pd.set_option('display.max_colwidth', 200)

In [3]:
# ▶️▶️
# Скачиваем библиотеку для загрузки файлов
!pip install wget --quiet

Команда %%time в начале ячейки замеряет время ее выполнения

In [4]:
%%time
# ▶️▶️
# Выполняется ≈1 минуту

# Скачиваем веса модели

from wget import download

# download('https://rusvectores.org/static/models/rusvectores4/news/news_upos_cbow_600_2_2018.vec.gz',
#         'news_upos_cbow_600_2_2018.vec.gz')

CPU times: total: 0 ns
Wall time: 0 ns


In [5]:
%%time
# ▶️▶️
# Выполняется ≈2-3 минуты
word2vec = KeyedVectors.load_word2vec_format('news_upos_cbow_600_2_2018.vec.gz')

CPU times: total: 1min 46s
Wall time: 3min 3s


In [6]:
# ✏️
# Атрибут index_to_key объекта word2vec содержит список слов,
# на котором обучалась модель

word2vec.index_to_key

['год_NOUN',
 'xx_NUM',
 'сообщать_VERB',
 'россия_PROPN',
 'мочь_VERB',
 'человек_NOUN',
 'российский_ADJ',
 'также_ADV',
 'время_NOUN',
 'новый_ADJ',
 'становиться_VERB',
 'xxxx_NUM',
 'заявлять_VERB',
 'страна_NOUN',
 'дело_NOUN',
 'область_NOUN',
 'первый_ADJ',
 'украина_PROPN',
 'компания_NOUN',
 'президент_NOUN',
 'рубль_NOUN',
 'отмечать_VERB',
 'слово_NOUN',
 'получать_VERB',
 'день_NOUN',
 'работа_NOUN',
 'глава_NOUN',
 'принимать_VERB',
 'xxx_NUM',
 'город_NOUN',
 'место_NOUN',
 'суд_NOUN',
 'решение_NOUN',
 'проводить_VERB',
 'вопрос_NOUN',
 'проходить_VERB',
 'москва_PROPN',
 'сказать_VERB',
 'результат_NOUN',
 'сша_PROPN',
 'район_NOUN',
 'тысяча_NOUN',
 'данные_NOUN',
 'представитель_NOUN',
 'ранее_ADV',
 'сегодня_ADV',
 'говорить_VERB',
 'напоминать_VERB',
 'отношение_NOUN',
 'матч_NOUN',
 'происходить_VERB',
 'миллион_NOUN',
 'команда_NOUN',
 'находиться_VERB',
 'информация_NOUN',
 'мир_NOUN',
 'составлять_VERB',
 'власть_NOUN',
 'организация_NOUN',
 'проект_NOUN',
 'пр

У нашей модели есть особенность: для ее обучения к словам цепляли [тэги с частью речи](https://universaldependencies.org/u/pos/all.html), например, 'год_NOUN', 'сообщать_VERB'.

Чтобы сопоставить обычное слово и слово с тэгом из модели, составим словарь переименования.

In [7]:
# ▶️▶️
# Метод split разбивает слово на список подслов по заданному символу

'год_NOUN'.split('_')

['год', 'NOUN']

In [8]:
# ▶️▶️
# В квадратных скобках из списка получаем начальный элемент (нумерация с 0)
'год_NOUN'.split('_')[0]

'год'

In [9]:
# ✏️
# Создадим словарь вида {слово: слово_ТЭГ}.
# В конце списка встречаются некорректные тэги, например, 'год_NOUN' 'год_PROPN'
# или ['давать_VERB', 'давать_NOUN', 'давать_ADJ', 'давать_PROPN', 'давать_NUM']
# Поэтому будем брать только первую версию слова, и если оно уже есть в словаре,
# перезаписывать его не будем

lemma2word = {}

for tagged_word in word2vec.index_to_key:
    word = tagged_word.split('_')[0]
    if word not in lemma2word:
        lemma2word[word] = tagged_word

In [10]:
# ▶️▶️
lemma2word['дать']

'дать_NOUN'

# Векторизация

Для того, чтобы иметь возможность автоматически анализировать данные, нужно придать тексту числовую структуру. Для этого сделаем векторизацию. Нам нужно пройти несколько стадий: токенизация → лемматизация → векторизация.

📝 **Токенизация** - процесс разбиения текста на слова (**токен** - минимальная единица язчка, имеющая смысл)

👨🏻‍💻 Вопрос: можно ли назвать букву токеном?

📝 **Лемматизация** - получение начальной формы слова: "годы", "году" → "год"

📝 **Векторизация** - процесс превращения текста в набор чисел - **векторное представление**. Например, "год" → (0.1, 0, 1.2, 3)

📝 **Вектор** - набор чисел, например, (0.1, 0, 1.2, 3). Над векторами можно проводить арифметические операции аналогично числам - например, попарное сложение элементов: (1, 2, 5) + (3, 4, 0) = (4, 6, 5) или вычисление расстояния между ними.

На основе близости векторных представлений текстов мы и будем искать похожие новости.

In [11]:
# ▶️▶️
# Пример токенизации

word_tokenize('Мороз и солнце, день чудесный', language='russian')

['Мороз', 'и', 'солнце', ',', 'день', 'чудесный']

In [12]:
# ▶️▶️
# mystem - инструмент для лемматизации
mystem = Mystem()

In [13]:
# ▶️▶️
# Функция получения тэгированной формы слова из модели Word2Vec

def get_w2v_word(word):
    # Получаем начальную форму слова
    lemma = mystem.lemmatize(word)[0]
    # Берем тэгированную форму из составленного ранее словаря
    w2v_word = lemma2word.get(lemma)
    return w2v_word

In [14]:
# ▶️▶️
get_w2v_word("Мороз")

'мороз_NOUN'

In [15]:
get_w2v_word("даю")

'давать_VERB'

In [16]:
# ▶️▶️
# Функция получения векторного представления слова из модели Word2Vec

def vectorize_sentence(txt):
    words = word_tokenize(txt, language='russian')
    vectors = []
    for word in words:
        tagged_word = get_w2v_word(word)
        if tagged_word is not None:
            vector = word2vec[tagged_word]
            vectors.append(vector)
    return np.mean(vectors, axis=0)

In [17]:
# ▶️▶️
vectorize_sentence('Мороз и солнце, день чудесный')

array([-3.57155018e-02, -3.17232534e-02,  2.36009993e-02,  2.01847516e-02,
       -5.54675004e-03, -2.23342497e-02,  3.79455015e-02,  1.16540007e-02,
       -3.00354995e-02,  2.97012478e-02,  2.92074997e-02, -4.01300099e-03,
        6.23749569e-04, -3.76459956e-02,  3.04417498e-02,  2.84999842e-05,
       -3.60299982e-02,  1.29974987e-02,  2.32912507e-02, -5.50955012e-02,
       -1.85812488e-02, -7.48549961e-03,  2.27707513e-02, -3.52999941e-03,
        3.59999985e-02, -7.51149980e-03, -2.02332512e-02, -4.89125028e-02,
        1.35750007e-02,  1.54049974e-03,  5.72499819e-04, -8.26525129e-03,
        3.06322500e-02, -1.44074997e-02, -3.37154977e-02, -1.13512501e-02,
        9.28924978e-03,  6.47550030e-03,  4.14189994e-02, -3.74612510e-02,
       -4.14294973e-02, -1.11744991e-02, -2.60677487e-02, -2.53522508e-02,
       -2.04227492e-02, -4.54100035e-03, -1.55797508e-02, -1.93010010e-02,
       -8.01124983e-03, -4.26614955e-02,  2.85055004e-02,  4.71442491e-02,
       -1.28867496e-02,  

# Возьмем данные о новостях с Lenta.ru

In [19]:
# ▶️▶️
data = pd.read_csv('https://raw.githubusercontent.com/anastasiarazb/skillbox_nlp_demo/master/lenta_example.csv',
                   sep=',')
data

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины российских биатлонистов,Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встреч...,Спорт,Зимние виды
1,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти на «сатанинское сборище» в Киеве,Архиепископ канонической Украинской православной церкви Московского патриархата (УПЦ МП) Лонгин отказался участвовать в «объединительном соборе» по приглашению Вселенского патриарха Варфоломея. Он...,Бывший СССР,Украина
2,https://lenta.ru/news/2018/12/15/ovechkin/,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал хет-трик в игре Национальной хоккейной лиги (НХЛ) с «Каролиной». Таким образом он продлил результативную серию до 13 матчей и повторил личный рекорд ...,Спорт,Хоккей
3,https://lenta.ru/news/2018/12/15/newyear/,Названы регионы России с самым дорогим и дешевым новогодним столом,"Производитель онлайн-касс «Эвотор» проанализировал данные со своих терминалов, установленных в тысячах продуктовых магазинов по стране, и рассчитал стоимость базового новогоднего стола в разных го...",Экономика,Деньги
4,https://lenta.ru/news/2018/12/15/gaz/,Россию и Украину пригласили на переговоры по газу,"Вице-президент Еврокомиссии Марош Шефчович пригласил Россию и Украину в Брюссель для следующего этапа трехсторонних переговоров по газу. Встреча состоится в январе, написал он в Twitter. «Ожидаем,...",Экономика,Госэкономика
...,...,...,...,...,...
24462,https://lenta.ru/news/2017/10/27/rynoktruda/,Десяти миллионам россиян предложили сменить профессию,"Россию ждет дефицит специалистов, способных выполнять аналитическую и творческую работу. К 2025 году нехватка таких кадров может достигнуть 10 миллионов человек. При этом столько же людей могут ок...",Экономика,Социальная сфера
24463,https://lenta.ru/news/2017/10/27/singsing/,Электромобили Nissan запоют ради пешеходов,"Японская автомобильная компания Nissan продемонстрировала на Токийском международном автосалоне звуковую систему Canto, разработанную специально для электромобилей. Видеопрезентация опубликована н...",Ценности,Движение
24464,https://lenta.ru/news/2017/10/27/red_russian/,Швейцарцы выпустили часы за 20 тысяч долларов в честь Октябрьской революции,"Швейцарская часовая компания H. Moser & Cie. представила новую модель часов Venturer XL Stoletniy Krasniy, сообщает Monochrome. Дизайн модели с темнеющим по краям красным циферблатом, надписью «Г....",Ценности,Часы
24465,https://lenta.ru/news/2017/10/27/crispr/,Создан эффективный способ лечения смертоносных мутаций,"Ученые из Массачусетского технологического института и Гарвардского университета разработали новый способ регулировки активности генов, который позволит лечить наследственные заболевания. Метод ос...",Наука и техника,Наука


In [20]:
# ✏️
# Выведем списки тем и тэгов со счетчиком числа статей в этих группах
# Функция groupby по списку колонок ['topic', 'tags'] группирует данные,
# а count() - дает количество записей в каждой группе

data.groupby(['topic', 'tags']).count()

url  title  text
topic           tags                               
Бывший СССР     undefined           46     46    46
                Белоруссия         162    162   162
                Все                  4      4     4
                Закавказье         245    245   245
                Молдавия            36     36    36
                Прибалтика         102    102   102
                Средняя Азия       117    117   117
                Украина           2548   2548  2548
Дом             Город              356    356   356
                Дача               236    236   236
                Квартира          1371   1371  1371
                Офис               125    125   125
Культура        undefined            2      2     2
                Все                  8      8     8
                Искусство          135    135   135
                Кино              1486   1486  1486
                Книги              124    124   124
                Музыка            1229   1229  1229
                Театр              171    171   171
                Фотография          16     16    16
Наука и техника Все                 11     11    11
                Гаджеты            198    198   198
                Жизнь              286    286   286
                Игры                35     35    35
                История            113    113   113
                Космос             699    699   699
                Наука              820    820   820
                Оружие            1144   1144  1144
                Софт                22     22    22
                Техника            183    183   183
Спорт           undefined          546    546   546
                Авто                62     62    62
                Бокс и ММА         682    682   682
                Все                 18     18    18
                Зимние виды        816    816   816
                Летние виды        247    247   247
                Футбол            2798   2798  2798
                Хоккей             289    289   289
Ценности        undefined            8      8     8
                Вкусы               37     37    37
                Внешний вид        310    310   310
                Все                  1      1     1
                Движение           150    150   150
                Инструменты        127    127   127
                Стиль              571    571   571
                Часы               135    135   135
                Явления            725    725   725
Экономика       undefined          199    199   199
                Бизнес             880    880   880
                Все                 40     40    40
                Госэкономика      2390   2390  2390
                Деловой климат     562    562   562
                Деньги             273    273   273
                Рынки              505    505   505
                Социальная сфера    66     66    66

In [28]:
# ✏️
# В теме 'Экономика' возьмем тэг 'undefined' и исследуем новости в нем

data_tag = data[
    (data['topic'] == 'Экономика')
    & (data['tags'] == 'undefined')
    ].copy()

# 👨🏻‍💻 Попробуйте другую комбинацию темы и тэга, напр. "Наука и техника"+"Техника"

data_tag.shape

(199, 5)

In [29]:
# ✏️

# Представим заголовки и тексты в виде чисел с помощью
# объединения векторных представлений, полученных с помощью
# vectorize_sentence для каждого в отдельности
# Операция + для двух списков объединяет их: [1, 2] + [3, 4] = [1, 2, 3, 4]

vectors = []

#for title, text in tqdm(data_tag[['title', 'text']].values):
for title, text in data_tag[['title', 'text']].values:
#    vector = vectorize_sentence(title) + vectorize_sentence(text)
    vector = vectorize_sentence(title)
    vectors.append(vector)

# 👨🏻‍💻 Попробуйте использовать только 1 элемент статьи, только текст
#    или только заголовок
#    (для этого в цикле надо будет оставить в vector, например,
#    только vectorize_sentence(text))

vectors = np.array(vectors)

# Кластеризация

📝 **Кластеризация** - объединение похожих объектов в группы.

Есть различные методы кластеризации, мы будем пользоваться библиотекой [sklearn](https://scikit-learn.org/stable/modules/clustering.html)

In [30]:
# ▶️▶️

from sklearn.cluster import SpectralClustering, KMeans

clusterizer = SpectralClustering(n_clusters=8, n_init=10, random_state=42)

# 👨🏻‍💻 Попробуйте другую модель кластеризации, например, KMeans.
#    Описание различных методов: https://scikit-learn.org/stable/modules/clustering.html

# 👨🏻‍💻 Попробуйте заменить параметры, например, установить n_clusters=10.
#    Есть ли какие-то идеи, как правильно выбрать количество кластеров?

clusterizer.fit(vectors)

SpectralClustering(random_state=42)

In [31]:
# ✏️
# Добавим разметку с номером кластера clusterizer.labels_
# как новую колонку в данные 'label' в таблицу data_tag
data_tag['label'] = clusterizer.labels_

In [32]:
data_tag.head(5)

,url,title,text,topic,tags,label
290,https://lenta.ru/news/2018/12/11/alphasell/,От Альфа-банка захотели избавиться,"Председатель совета директоров «Альфа Групп» Михаил Фридман ведет переговоры о продаже Альфа-банка. Как сообщает Financial Times со ссылкой на источники, переговоры шли, в частности, с ВТБ и UniCr...",Экономика,undefined,6
341,https://lenta.ru/news/2018/12/11/alfa_oproverzhenie/,«Альфа Групп» опровергла сообщения о продаже активов,«Альфа Групп» официально опровергала появившиеся сообщения о продаже своих активов. Об этом говорится в пресс-релизе группы. «В связи с появлением в СМИ сообщений о переговорах по продаже банковск...,Экономика,undefined,3
360,https://lenta.ru/news/2018/12/10/pensia/,Раскрыты размеры пенсий в России,"В Пенсионном фонде России (ПФР) рассказали о размерах пенсий в России. Об этом сообщает «Российская газета». Согласно данным, самая высокая пенсия в России выплачивается на Чукотке. Ее средний раз...",Экономика,undefined,7
546,https://lenta.ru/news/2018/12/07/yugra/,Стали известны новые подробности в деле «Югры»,"В деле, предваряющем отзыв лицензии банка «Югра», появились новые подробности. В редакцию «Ленты.ру» поступили документы, из которых следует, что предыдущие владельцы и администрация банка предлаг...",Экономика,undefined,3
594,https://lenta.ru/news/2018/12/06/pribavka/,Ветеран труда отправил Медведеву прибавку к пенсии,Ветеран труда из Магадана отказался от назначенной ему областными властями доплаты в размере 683 рублей 66 копеек и предложил перечислить эти деньги премьер-министру Дмитрию Медведеву. Документ о ...,Экономика,undefined,7


In [35]:
# ✏️

# Набор меток data_tag['label'].unique() отсортируем через sorted()
# и для каждой метки выведем label и подтаблицу строк с этой
# меткой кластера data_tag[data_tag['label'] == label]

for label in sorted(data_tag['label'].unique()):
    print(label)
    display(data_tag[data_tag['label'] == label])

0


,url,title,text,topic,tags,label
2290,https://lenta.ru/news/2018/11/09/oikforeuro/,Нефтяники решили защититься от санкций,"Российские компании «Газпромнефть» и «Сургутнефтегаз» ведут ожесточенные переговоры с западными покупателями, чтобы добиться пересмотра контрактов на 2019 год. Об этом сообщает Reuters со ссылкой ...",Экономика,undefined,0
2743,https://lenta.ru/news/2018/11/01/gasukraine/,«Нафтогаз» уличили в закупках российского газа,"«Нафтогаз Украины» может оказаться потребителем газа, реализуемого на электронной торговой площадке (ЭТП) компанией «Газпром экспорт», выяснил «Коммерсантъ». По данным за первые полтора месяца раб...",Экономика,undefined,0
3282,https://lenta.ru/news/2018/10/24/rossneft_eni/,«Роснефть» и Eni обсудили пути сотрудничества в условиях американских санкций,"Российская «Ронефть» и итальянская энергетическая компания Eni обсуждают формы своего сотрудничества в условиях санкций, введенных США. Компании ставят целью продолжить совместные проекты, не нару...",Экономика,undefined,0
3643,https://lenta.ru/news/2018/10/17/polskagaz/,Польша пересела на газ из США,"Польша будет закупать сжиженный природный газ (СПГ) из США по цене почти на 30 процентов меньше, чем ей сейчас обходится природный газ из России. Об этом заявил глава польской нефтегазовой компани...",Экономика,undefined,0
3676,https://lenta.ru/news/2018/10/17/nordstream3/,Россия задумалась о «Северном потоке-3»,"Российские власти допускают строительство трубопровода «Северный поток-3». Как сообщил ТАСС заместитель министра энергетики Анатолий Яновский, новый проект могут запустить в случае роста спроса на...",Экономика,undefined,0
4609,https://lenta.ru/news/2018/10/02/ukroil/,Украина начала запасаться нефтью для чрезвычайных ситуаций,"На Украине решили создать стратегический запас нефти и нефтепродуктов на случай чрезвычайных ситуаций, сообщает ТАСС со ссылкой на председателя Государственного агентства резерва Вадима Мосийчука....",Экономика,undefined,0
6259,https://lenta.ru/news/2018/09/04/shokhin/,Бизнес отреагировал на предложение Путина защитить предпенсионеров,"Чрезмерно жесткая ответственность за увольнение граждан предпенсионного возраста приведет к тому, что работу будут терять сотрудники, не достигшие этого возраста. Об этом заявил журналистам глава ...",Экономика,undefined,0
6815,https://lenta.ru/news/2018/08/24/merkeligruzin/,Германия согласилась на «зависимость» от России,"Газопровод «Северный поток-2» не заставит Германию зависеть от России, заявила канцлер Германии Ангела Меркель, выступая перед студентами в Тбилиси, передает Reuters. «Мы уже видим, что это не при...",Экономика,undefined,0
6901,https://lenta.ru/news/2018/08/23/creditsuisseoproverg/,Credit Suisse открестился от заморозки российских денег,"Швейцарский банк Credit Suisse опроверг информацию о заморозке счетов российских клиентов. Как сообщает РБК со ссылкой на пресс-службу банка, речь идет не о заморозке, а о реклассификации определе...",Экономика,undefined,0
6991,https://lenta.ru/news/2018/08/22/azergas/,Германия заинтересовалась газопроводом в обход России,"Канцлер Германии Ангела Меркель в ходе визита в Азербайджан планирует обсудить перспективы поставок каспийского газа в Европу. Как сообщает Reuters, в ходе официальных встреч стороны рассмотрят во...",Экономика,undefined,0


1


,url,title,text,topic,tags,label
1195,https://lenta.ru/news/2018/11/27/block/,Банки пригрозили блокировать карты россиян за необоснованные переводы,"Банки начали запрашивать у своих клиентов подтверждение операций на небольшие суммы. Если человек не предоставит обоснование перевода, карту могут аннулировать, сообщают «Известия». Клиенты Бинбан...",Экономика,undefined,1
1291,https://lenta.ru/news/2018/11/26/shortdollar/,Россияне вывели с валютных счетов миллиарды долларов,"Объем долгосрочных вкладов в валюте за октябрь в российских банках сократился на 300 миллиардов рублей (4,5 миллиарда долларов) передает РБК со ссылкой на данные Банка России. Объем краткосрочных ...",Экономика,undefined,1
2672,https://lenta.ru/news/2018/11/02/dolg_record/,Россияне влезли в рекордные долги,"Выдачи потребительских кредитов в России в третьем квартале 2018 года достигли исторического максимума. Об этом говорится в исследовании БКИ «Эквифакс», которое имеется в распоряжении «Ленты.ру». ...",Экономика,undefined,1
2877,https://lenta.ru/news/2018/10/30/credit/,Россияне набрали рекордных кредитов для погашения кредитов,"Россияне за первое полугодие 2018 года набрали новых кредитов для покрытия старых на 68,3 миллиарда рублей. Это превышает показатели за аналогичный период прошлого года в 1,7 раза, сообщает РИА Но...",Экономика,undefined,1
3286,https://lenta.ru/news/2018/10/24/fake_money/,Россиян предупредили о новых поддельных купюрах,"Фальшивомонетчики начали подделывать новые банкноты номиналом 200 рублей, следует из отчета Центробанка. Так, в третьем квартале 2018 года были обнаружены три первые подделки. О фальшивых 2000-руб...",Экономика,undefined,1
4146,https://lenta.ru/news/2018/10/09/bankpolicy/,Банки помогут государству следить за россиянами,"Федеральная налоговая служба (ФНС) сможет получать в банках максимально полную информацию о счетах налогоплательщиков (граждан, ИП и компаний), сообщает газета «Коммерсантъ». Новый порядок оформле...",Экономика,undefined,1
4170,https://lenta.ru/news/2018/10/09/cbdollar/,Россияне начали забирать деньги из банков,"Последние два месяца Центробанк отмечает сокращение объема банковских вкладов. В августе отток денег составил 0,6 процента, а в сентябре — 0,8 процента. Об этом сообщает РБК со ссылкой на зампреда...",Экономика,undefined,1
5417,https://lenta.ru/news/2018/09/18/kognitivedissonanz/,Доллары россиян пообещали сохранить,"Конвертация долларовых вкладов в рубли не рассматривалась и не рассматривается «ни в каких даже самых страшных сценариях». Об этом заявил глава Сбербанка Герман Греф, передает РБК. «Что касается с...",Экономика,undefined,1
5614,https://lenta.ru/news/2018/09/14/guardiansofdollars/,Банк России пообещал защитить доллары россиян,"В Центробанке (ЦБ) не исключают, что в рамках нового пакета санкций власти США фактически запретят российским банкам пользоваться долларами. Об этом заявила глава Банка России Эльвира Набиуллина, ...",Экономика,undefined,1
6167,https://lenta.ru/news/2018/09/05/bankmoney/,Банки призвали воздержаться от пятитысячных купюр,"Компания NCR, один из крупнейших производителей банковского оборудования, в течение недели поставит в российские банки новую прошивку для своих банкоматов. Как сообщает принадлежащая семье Демьяна...",Экономика,undefined,1


2


...

,url,title,text,topic,tags,label
1558,https://lenta.ru/news/2018/11/22/ipkpens/,Пенсионный фонд отреагировал на заморозку пенсионных накоплений,"Закон об индивидуальном пенсионном капитале (ИПК) должен окончательно прояснить вопрос о накопительной части пенсии, передает ТАСС слова главы Пенсионного фонда России (ПФР) Антона Дроздова. Дрозд...",Экономика,undefined,5
1734,https://lenta.ru/news/2018/11/19/cbzapret/,ЦБ ограничил обмен валюты для системы денежных переводов,"Банк России на полгода ограничил системе денежных переводов «Юнистрим» наличные валютообменные операции, сообщает газета «Коммерсантъ». Ограничение введено за нарушения антиотмывочного законодател...",Экономика,undefined,5
2678,https://lenta.ru/news/2018/11/02/ztltnayaekonomika/,В Москве обсудили переход к низкоуглеродной модели экономики,"Российское партнерство за сохранение климата и Ассоциация менеджеров России провели открытую встречу с исполнительным директором Центра по эффективному использованию энергии, доктором экономически...",Экономика,undefined,5
3237,https://lenta.ru/news/2018/10/24/cbmfo/,ЦБ захотел снизить ставки по кредитам,"Банк России предлагает ограничить ставку по кредитам и займам 1 процентом в день. Как сообщает РБК, эти нормы содержатся в подготовленных регулятором поправках в законы «О потребительском кредите ...",Экономика,undefined,5
3831,https://lenta.ru/news/2018/10/15/yugra_svr/,Обнародованы документы об устойчивости «Югры» до ввода временной администрации,"14 октября на сайте Союза вкладчиков были обнародованы документы, названные в заголовке публикации «сенсационными». Цель данной публикации — предоставить общественности и правоохранительным орган...",Экономика,undefined,5
3923,https://lenta.ru/news/2018/10/12/pensiasud/,Пенсионный фонд ополчился на бизнес,"Пенсионный фонд России (ПФР) стал активнее судиться с компаниями за страховые взносы. Как подсчитала принадлежащая семье Демьяна Кудрявцева газета «Ведомости», за девять месяцев 2018 года он подал...",Экономика,undefined,5
3929,https://lenta.ru/news/2018/10/12/holdings/,Ставки по банковским вкладам выросли,"Сбербанк второй раз за год повысил ставки по всем рублевым вкладам, передает РБК. Также о повышении ставок заявили в ВТБ. Так, диапазон ставок Сбербанка на сегодняшний день составил от 3,4 до 5,15...",Экономика,undefined,5
4141,https://lenta.ru/news/2018/10/09/moneypens/,На повышении пенсионного возраста сэкономят триллионы,"Уже в 2019 году за счет повышения пенсионного возраста планируется сэкономить около 90 миллиардов рублей, сообщил в интервью «Известиям» глава Пенсионного фонда России Антон Дроздов. За шесть лет ...",Экономика,undefined,5
5177,https://lenta.ru/news/2018/09/21/bankbank/,ЦБ тайно перевел миллиарды в Крым,Банк России перевел работающему в Крыму Генбанку порядка 20 миллиардов рублей. Как сообщает принадлежащая семье Демьяна Кудрявцева газета «Ведомости» со ссылкой на аналитика Fitch Александра Данил...,Экономика,undefined,5
5686,https://lenta.ru/news/2018/09/13/imfseniors/,МВФ поддержал повышение пенсионного возраста в России,Пенсионную реформу поддержали исполнительные директора Международного валютного фонда (МВФ). Об этом сообщает РБК со ссылкой на заключительное заявления МВФ по завершении консультаций с Россией. М...,Экономика,undefined,5


6


,url,title,text,topic,tags,label
290,https://lenta.ru/news/2018/12/11/alphasell/,От Альфа-банка захотели избавиться,"Председатель совета директоров «Альфа Групп» Михаил Фридман ведет переговоры о продаже Альфа-банка. Как сообщает Financial Times со ссылкой на источники, переговоры шли, в частности, с ВТБ и UniCr...",Экономика,undefined,6
703,https://lenta.ru/news/2018/12/05/penscredit/,Сбербанк предложил пенсионерам взять кредиты,"Сбербанк поднял максимальный возраст, при котором можно рассчитывать на кредит, до 70 лет. Об этом говорится в официальном сообщении банка, поступившем в «Ленту.ру». Потребительские кредиты без об...",Экономика,undefined,6
898,https://lenta.ru/news/2018/12/02/sber/,Вкладчики Сбербанка получат меньше денег,"С 1 декабря максимальная ставка для розничных вкладов Сбербанка снизилась на 0,15 процентного пункта, сообщает принадлежащая семье Демьяна Кудрявцева газета «Ведомости». Банк закрыл свой самый выг...",Экономика,undefined,6
2323,https://lenta.ru/news/2018/11/09/sber_moved/,Сбербанк переедет,"Сбербанк планирует открыть новую штаб-квартиру в районе «Москва-Сити» к 2021 году, передает ТАСС. Отмечается, что банк решил перенести главный офис на улицу Кульнева у станции метро «Кутузовская» ...",Экономика,undefined,6
2972,https://lenta.ru/news/2018/10/29/sberutek/,Сбербанк столкнулся с масштабной утечкой,"Имена и адреса электронной почты примерно 421 тысячи сотрудников Сбербанка попали в сеть, сообщает газета «Коммерсантъ». Базу данных сотрудников Сбербанка выложили на специализированном форуме phr...",Экономика,undefined,6
4358,https://lenta.ru/news/2018/10/05/biobank/,Сбербанку понадобились биометрические данные клиентов,"Сбербанк начал собирать биометрические данные клиентов на собственной платформе. Как сообщает РИА Новости, клиенты могут сдать голосовые данные в отделении банка или через call-центр. По словам пр...",Экономика,undefined,6
6269,https://lenta.ru/news/2018/09/04/sber_stoek/,Банкоматы Сбербанка продолжили принимать наличные без ограничений,"Сбербанк не стал вводить ограничения на прием купюр в банкоматах, несмотря на предупреждение о массовом вбросе фальшивых пятитысячных банкнот в аппараты, сообщает ТАСС со ссылкой на пресс-службу к...",Экономика,undefined,6
6904,https://lenta.ru/news/2018/08/23/sber_davos/,Сбербанк и Давосский форум договорились бороться с киберпреступностью,"Председатель правления Сбербанка Герман Греф и основатель Давосского всемирного экономического форума, профессор Клаус Шваб подписали партнерское соглашение в рамках Центра кибербезопасности Всеми...",Экономика,undefined,6
12332,https://lenta.ru/news/2018/05/22/sber_turk/,Сбербанк продал свою «дочку» арабам,"Сбербанк договорился о продаже своей турецкой «дочки» Denizbank банку Emirates NBD из Объединенных Арабских Эмиратов за 3,2 миллиарда долларов. Об этом во вторник, 22 мая, говорится в совместном с...",Экономика,undefined,6
17135,https://lenta.ru/news/2018/02/26/sber_robot/,Сбербанк заменил людей роботами,"Сбербанк приступил к роботизации контактного центра, теперь на вопросы корпоративных клиентов будет отвечать робот по имени Анна. Об этом в понедельник, 26 февраля, сообщает пресс-служба кредитной...",Экономика,undefined,6


7


,url,title,text,topic,tags,label
360,https://lenta.ru/news/2018/12/10/pensia/,Раскрыты размеры пенсий в России,"В Пенсионном фонде России (ПФР) рассказали о размерах пенсий в России. Об этом сообщает «Российская газета». Согласно данным, самая высокая пенсия в России выплачивается на Чукотке. Ее средний раз...",Экономика,undefined,7
594,https://lenta.ru/news/2018/12/06/pribavka/,Ветеран труда отправил Медведеву прибавку к пенсии,Ветеран труда из Магадана отказался от назначенной ему областными властями доплаты в размере 683 рублей 66 копеек и предложил перечислить эти деньги премьер-министру Дмитрию Медведеву. Документ о ...,Экономика,undefined,7
3821,https://lenta.ru/news/2018/10/15/pred/,Предпенсионеров приравняют к студентам,"Российских предпенсионеров будут переобучать в рамках программы повышения квалификации и платить стипендию. Такие предложения содержатся в проекте федеральной программы за авторством Минтруда, выд...",Экономика,undefined,7
4644,https://lenta.ru/news/2018/10/01/soglasie/,Россиян переведут на накопительную пенсию без их согласия,"Россиян переведут на новую систему накопительной пенсии без их согласия. Об этом РБК сообщили два федеральных чиновника, эксперт, знакомый с обсуждением законопроекта, и источник на финансовом рын...",Экономика,undefined,7
5826,https://lenta.ru/news/2018/09/11/familybusiness/,Пенсионеров задумали превратить в бизнесменов,"Пенсионеров планируют вовлекать в семейный бизнес. Как сообщил в интервью ТАСС глава Торгово-промышленной палаты России Сергей Катырин на Восточном экономическом форуме (ВЭФ), для этого, в частнос...",Экономика,undefined,7
5920,https://lenta.ru/news/2018/09/10/dobrovolodin/,Стала известна судьба повышенных пенсий депутатов,"Законопроект о добровольном отказе депутатов от повышенной пенсии поддержал председатель Госдумы Вячеслав Володин, передает ТАСС. «Законопроект, думаю, я и мои коллеги однозначно поддержим», — соо...",Экономика,undefined,7
6196,https://lenta.ru/news/2018/09/05/atomicoldmen/,Атомщики отказались рано уходить на пенсию,"Почти половина сотрудников атомной отрасли — люди предпенсионного и пенсионного возраста, которые не спешат уходить на пенсию, сообщил ТАСС глава «Росатома» Алексей Лихачев. По его подсчетам, таки...",Экономика,undefined,7
6607,https://lenta.ru/news/2018/08/29/pensy/,Россиян предпенсионного возраста приравняли к беременным,"Работодателям за увольнение сотрудников предпенсионного возраста могут грозить такие же штрафы, как при увольнении по мотивам беременности, передает РИА Новости со ссылкой на вице-премьера Татьяну...",Экономика,undefined,7
8250,https://lenta.ru/news/2018/07/31/pensions_up/,Работающим пенсионерам повысят выплаты,"С 1 августа повысится размер пенсий работающих пенсионеров, сообщает ТАСС со ссылкой на пресс-службу Пенсионного фонда. Максимальная прибавка составит 235 рублей. «Размер прибавки рассчитывается и...",Экономика,undefined,7
18385,https://lenta.ru/news/2018/02/07/gref/,Греф поностальгировал о своей работе дворником,"Председатель правления ПАО «Сбербанк России» Герман Греф на встрече с участниками финала конкурса «Лидеры России» рассказал о своей работе дворником. Об этом сообщает «Интерфакс». «Я помню, когда ...",Экономика,undefined,7


# Что мы узнали на этом занятии?

ML Engineer - это специалист, который строит модели машинного обучения, то есть системы, которые создают "магию" интеллектуального поведения

Это творческая профессия, где каждый случай нужно исследовать

Мы разобрали, как работает нейронная сеть

Научились переводить человеческий текст в численное представление (векторы) и объединять их в группы с помощью методов кластеризации

Все необходимые для работы функции уже реализованы в библиотеках. Мы познакомились с библиотеками gensim, nltk, pymystem, sklearn